# Global Temperature Analysis
Abinash Patti

### Objective
Analyze global temperature data to identify global warming trends, regional anomolies, and seasonal patterns

### Dataset
Source: Berkeley Earth / Kaggle

Files include temperature readings by city, country, and globally from 1750-2015

In [2]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Load GlobalLandTemperaturesByCountry into a DataFrame
df_global_land_temps_country = pd.read_csv("../data/GlobalLandTemperaturesByCountry.csv")

# Display the info and first few rows
df_global_land_temps_country.info()
df_global_land_temps_country.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577462 entries, 0 to 577461
Data columns (total 4 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   dt                             577462 non-null  object 
 1   AverageTemperature             544811 non-null  float64
 2   AverageTemperatureUncertainty  545550 non-null  float64
 3   Country                        577462 non-null  object 
dtypes: float64(2), object(2)
memory usage: 17.6+ MB


,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


### Average Temperature Summaries

In [16]:
# Find special summaries
mean_temp = df_global_land_temps_country["AverageTemperature"].mean()
std_temp = df_global_land_temps_country["AverageTemperature"].std()

max_temp = df_global_land_temps_country["AverageTemperature"].max()
min_temp = df_global_land_temps_country["AverageTemperature"].min()

max_temp_row = df_global_land_temps_country[df_global_land_temps_country["AverageTemperature"] == max_temp]
min_temp_row = df_global_land_temps_country[df_global_land_temps_country["AverageTemperature"] == min_temp]

# Display
print("Mean temperature (1750 - 2015): ", round(mean_temp, 2))
print("Standard deviation (1750 - 2015): ", round(std_temp, 2))
print("Highest temperature: ", round(max_temp, 2))
print("Lowest temperature: ", round(min_temp, 2))

Mean temperature (1750 - 2015):  17.19
Standard deviation (1750 - 2015):  10.95
Highest temperature:  38.84
Lowest temperature:  -37.66


In [21]:
# Convert dt col to datetime
df_global_land_temps_country["dt"] = pd.to_datetime(df_global_land_temps_country["dt"])

# Extract year to new column
df_global_land_temps_country["Year"] = df_global_land_temps_country["dt"].dt.year

# Confirm with head
df_global_land_temps_country.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country,Year
0,1743-11-01,4.384,2.294,Åland,1743
1,1743-12-01,NaN,NaN,Åland,1743
2,1744-01-01,NaN,NaN,Åland,1744
3,1744-02-01,NaN,NaN,Åland,1744
4,1744-03-01,NaN,NaN,Åland,1744


### Extreme Temperatures by Country

In [ ]:
# Group by country
df_country_group = df_global_land_temps_country.groupby("Country")

# List to hold summary
list_summary_extreme_temps = []
